In [ ]:
-- create api integration using Frosty Fridays as the allowed prefix;
-- this way multiple git repository stages can be built from a single api integration

CREATE API INTEGRATION IF NOT EXISTS FROSTY_FRIDAY_GIT
api_provider = git_https_api
api_allowed_prefixes = ('https://github.com/FrostyFridays/')
enabled = true

In [ ]:
-- create the git repository stage specifically for FF_week_91

CREATE GIT REPOSITORY IF NOT EXISTS WEEK_91_REPO
ORIGIN = 'https://github.com/FrostyFridays/FF_week_91.git'
API_INTEGRATION = frosty_friday_git

In [ ]:
-- see what branches are in the repo
-- extraneous for this challenge but useful for exploration

show git branches in week_91_repo

In [ ]:
-- see what files are in the main branch
-- extraneous for this challenge but useful for exploration

ls @week_91_repo/branches/main

In [ ]:
-- create sample table, code directly from the challenge

CREATE OR REPLACE TABLE people (
id INTEGER,
name VARCHAR
);

INSERT INTO people (id, name) VALUES
(1, 'Alice'),
(2, 'Bob'),
(3, 'Charlie'),
(4, 'David'),
(5, 'Eva'),
(6, 'Fiona'),
(7, 'George'),
(8, 'Hannah'),
(9, 'Ian'),
(10, 'Julia');

In [ ]:
-- create a prodecure that references the function within the python script
-- parameters of prodecure should match python function

CREATE OR REPLACE PROCEDURE add_one_to_column(tableName VARCHAR, columnName VARCHAR)
  RETURNS TABLE()
  LANGUAGE PYTHON
  RUNTIME_VERSION = '3.8'
  PACKAGES = ('snowflake-snowpark-python')
  IMPORTS = ('@FROSTY_FRIDAY.WEEK_91.WEEK_91_REPO/branches/main/add_1.py')
  HANDLER = 'add_1.add_one_to_column';

In [ ]:
-- call the procedure on the sample table
-- important to note the proc doesn't change the source data
-- it just performs the operation on the column and returns the table
-- similar to running a select on the table and adding one to a column

call add_one_to_column('people','id')

In [ ]:
-- select * from the table shows no change to underlying table

select * from people

In [ ]:
# not for running, just for reference

from snowflake.snowpark.functions import col

def add_one_to_column(session, table_name, column_name):
    df = session.table(table_name)
    updated_df = df.withColumn(column_name, col(column_name) + 1)
    return updated_df
